In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_table('./data/round1_ijcai_18_train_20180301.txt', sep=' ')
data.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


In [3]:
user_data = data[['user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level', 'is_trade']]
user_data.head()

,user_id,user_gender_id,user_age_level,user_occupation_id,user_star_level,is_trade
0,4505772604969228686,1,1003,2005,3003,0
1,2692638157208937547,0,1002,2005,3006,0
2,5247924392014515924,0,1003,2005,3004,0
3,2681414445369714628,1,1004,2005,3006,0
4,2729475788342039013,0,1002,2005,3001,0


In [4]:
def feature_group(data, feature, is_trade):
    if (is_trade != 1 and is_trade != 0):
        return None
    else:
        data = data[[feature, 'is_trade']]
        data_feature = data[data['is_trade'] == is_trade]
        data_feature = data_feature.reset_index(drop=True)
        data_feature_grouped = data_feature.groupby([feature]).count()
        data_feature_grouped = data_feature_grouped.reset_index()
        data_feature_grouped = data_feature_grouped.sort_values(['is_trade'], ascending=False)
    return data_feature_grouped

In [5]:
def feature_log_cal(data_feature_grouped, feature, is_trade):
    if (is_trade != 1 and is_trade != 0):
        return None
    else:
        new_feature = feature + '_' + str(is_trade)
        data_feature_grouped[new_feature] = np.log(data_feature_grouped['is_trade'] / len(data_feature_grouped))
#         # 0-1标准化
#         data_feature_grouped[new_feature] =  (data_feature_grouped['is_trade'] - data_feature_grouped['is_trade'].min())/\
#                                         (data_feature_grouped['is_trade'].max() - data_feature_grouped['is_trade'].min())
    return data_feature_grouped

# 1.user_gender_id

In [6]:
user_gender_id_grouped_1 = feature_group(user_data, 'user_gender_id', 1)
user_gender_id_grouped_1 = feature_log_cal(user_gender_id_grouped_1, 'user_gender_id', 1)
user_gender_id_grouped_1

,user_gender_id,is_trade,user_gender_id_1
1,0,6666,7.418481
2,1,1994,6.211604
3,2,195,3.886705
0,-1,166,3.725693


In [7]:
user_gender_id_grouped_0 = feature_group(user_data, 'user_gender_id', 0)
user_gender_id_grouped_0 = feature_log_cal(user_gender_id_grouped_0, 'user_gender_id', 0)
user_gender_id_grouped_0

,user_gender_id,is_trade,user_gender_id_0
1,0,354151,11.391184
2,1,92076,10.044075
0,-1,12736,8.065894
3,2,10154,7.839329


In [8]:
user_gender_id_grouped_1.to_csv('./feature_data/user_data/user_gender_id_grouped_1.txt', sep=' ', index=False, encoding='utf-8')
user_gender_id_grouped_0.to_csv('./feature_data/user_data/user_gender_id_grouped_0.txt', sep=' ', index=False, encoding='utf-8')

# 2. user_age_level

In [9]:
user_age_level_grouped_1 = feature_group(user_data, 'user_age_level', 1)
user_age_level_grouped_1 = feature_log_cal(user_age_level_grouped_1, 'user_age_level', 1)
user_age_level_grouped_1

,user_age_level,is_trade,user_age_level_1
4,1003,3005,5.810808
5,1004,2520,5.634790
6,1005,1340,5.003200
3,1002,1114,4.818488
7,1006,647,4.275122
1,1000,177,2.978925
8,1007,121,2.598566
2,1001,85,2.245427
0,-1,12,0.287682


In [10]:
user_age_level_grouped_0 = feature_group(user_data, 'user_age_level', 0)
user_age_level_grouped_0 = feature_log_cal(user_age_level_grouped_0, 'user_age_level', 0)
user_age_level_grouped_0

,user_age_level,is_trade,user_age_level_0
4,1003,166444,9.825190
5,1004,123611,9.527670
3,1002,69061,8.945521
6,1005,55268,8.722725
7,1006,29543,8.096378
1,1000,12781,7.258490
2,1001,6361,6.560716
8,1007,5096,6.338987
0,-1,952,4.661340


In [11]:
user_age_level_grouped_1.to_csv('./feature_data/user_data/user_age_level_grouped_1.txt', sep=' ', index=False, encoding='utf-8')
user_age_level_grouped_0.to_csv('./feature_data/user_data/user_age_level_grouped_0.txt', sep=' ', index=False, encoding='utf-8')

# 3. user_occupation_id

In [12]:
user_occupation_id_grouped_1 = feature_group(user_data, 'user_occupation_id', 1)
user_occupation_id_grouped_1 = feature_log_cal(user_occupation_id_grouped_1, 'user_occupation_id', 1)
user_occupation_id_grouped_1

,user_occupation_id,is_trade,user_occupation_id_1
4,2005,5498,7.002702
1,2002,2985,6.391917
3,2004,440,4.477337
2,2003,86,2.844909
0,-1,12,0.875469


In [13]:
user_occupation_id_grouped_0 = feature_group(user_data, 'user_occupation_id', 0)
user_occupation_id_grouped_0 = feature_log_cal(user_occupation_id_grouped_0, 'user_occupation_id', 0)
user_occupation_id_grouped_0

,user_occupation_id,is_trade,user_occupation_id_0
4,2005,294538,10.983725
1,2002,143932,10.267658
3,2004,23363,8.449471
2,2003,6332,7.143934
0,-1,952,5.249127


In [14]:
user_occupation_id_grouped_1.to_csv('./feature_data/user_data/user_occupation_id_grouped_1.txt', sep=' ', index=False, encoding='utf-8')
user_occupation_id_grouped_0.to_csv('./feature_data/user_data/user_occupation_id_grouped_0.txt', sep=' ', index=False, encoding='utf-8')

# 4. user_star_level（看成非类别型变量）

In [15]:
user_star_level_grouped_1 = feature_group(user_data, 'user_star_level', 1)
user_star_level_grouped_1 = feature_log_cal(user_star_level_grouped_1, 'user_star_level', 1)
# user_star_level_grouped_1['user_star_level'] = user_star_level_grouped_1['user_star_level'].apply(lambda x: x % 3000)
# user_star_level_grouped_1.loc[0, 'user_star_level'] = -1
user_star_level_grouped_1

,user_star_level,is_trade,user_star_level_1
7,3006,1575,4.877104
4,3003,1449,4.793722
6,3005,1263,4.656338
3,3002,1246,4.642787
5,3004,1217,4.619237
8,3007,1049,4.470686
9,3008,392,3.486355
1,3000,374,3.439349
2,3001,349,3.370165
10,3009,90,2.014903


In [16]:
user_star_level_grouped_0 = feature_group(user_data, 'user_star_level', 0)
user_star_level_grouped_0 = feature_log_cal(user_star_level_grouped_0, 'user_star_level', 0)
# user_star_level_grouped_0['user_star_level'] = user_star_level_grouped_0['user_star_level'].apply(lambda x: x % 3000)
# user_star_level_grouped_0.loc[0, 'user_star_level'] = -1
user_star_level_grouped_0

,user_star_level,is_trade,user_star_level_0
7,3006,80434,8.810286
4,3003,73129,8.715074
6,3005,66177,8.615182
3,3002,61675,8.544727
5,3004,60552,8.526351
8,3007,52266,8.379195
1,3000,30962,7.855609
9,3008,19786,7.407823
2,3001,18976,7.366024
10,3009,4034,5.817607


In [17]:
user_star_level_grouped_1.to_csv('./feature_data/user_data/user_star_level_grouped_1.txt', sep=' ', index=False, encoding='utf-8')
user_star_level_grouped_0.to_csv('./feature_data/user_data/user_star_level_grouped_0.txt', sep=' ', index=False, encoding='utf-8')